In [17]:
import faiss
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from tqdm import tqdm
tqdm.pandas(mininterval=1.0)
from TweetNormalizer import normalizeTweet


In [15]:

# Read the CSV file into a pandas DataFrame
# Replace 'your_file.csv' with the actual path to your CSV file
df = pd.read_csv('sample_200k-130k_text.csv')  # Assuming the texts are in a column named 'text'

# Check the first few rows to inspect the data and find the column with texts
df["text"].to_list()


['📊Interesting chart: we can clearly see how long-term holders been accumulating Bitcoins during the Summer-Autumn dump.   At the moment total supply held by holders is at almost All Time High.#Btc #100xgem #cryptocurrencies #HODL #ToTheMoon https://t.co/NusqceFbnv',
 "Bitcoin's 'Elon Musk pump' rally to $48K was exclusively driven by whales https://t.co/F6zGnXnBh0 Laranjal do Elon",
 "#Bitcoin over $50K this morning...is this the #DeadCatBounce that will go parabolic? Yesterday, $SQ announced they purchased Bitcoin...did #TSLA buy more in the 40s too? I continue to be bullish on Bitcoin which is why I'm bullish on $MARA, $RIOT, $MSTR, and $TSLA.",
 'Who cares about China’s ban on #Bitcoin.   Can’t ban crypto - moving on....',
 'bitcoin tries to take direction again. ⬆️⬇️ I think after some correction it will be up again. $btc #btc 💬',
 'The # Skyllz crew is creating the blockchain based and distributed skill validation platform to showcase track and boost human talents acro',
 'Saying

In [16]:
df["text"] = df["text"].progress_apply(normalizeTweet)

  0%|          | 0/129991 [00:00<?, ?it/s]

100%|██████████| 129991/129991 [00:28<00:00, 4584.90it/s]


In [18]:

# Function to preprocess text: remove punctuation and convert to lowercase
def preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    return text.lower()  # Convert text to lowercase


In [19]:

# Assuming the texts are in a column named 'text' (adjust if your column is named differently)
texts = df['text'].apply(preprocess).tolist()  # Preprocess all texts


In [20]:

# Vectorize the texts using TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

# Convert the TF-IDF matrix to a dense numpy array (FAISS requires dense data)
dense_matrix = tfidf_matrix.toarray().astype('float32')


MemoryError: Unable to allocate 112. GiB for an array with shape (129991, 115759) and data type float64

In [ ]:

# Create FAISS index (using L2 distance for similarity)
index = faiss.IndexFlatL2(dense_matrix.shape[1])  # L2 distance is the most common choice for cosine similarity
index.add(dense_matrix)  # Add all the vectors to the index

# Define the threshold for similarity (e.g., 0.85 means 85% similarity)
threshold = 0.85
k_nearest_neighbors = 10  # Number of nearest neighbors to check for similarity

# Perform similarity search for each text
similar_texts = set()  # Set to track indices of similar texts

for i in range(len(dense_matrix)):
    # Search for the k nearest neighbors of text i
    distances, indices = index.search(dense_matrix[i:i+1], k_nearest_neighbors)  # Search 1 text at a time
    
    for idx, dist in zip(indices[0], distances[0]):
        if dist < (1 - threshold):  # If similarity is above threshold (cosine similarity is 1 - L2 distance)
            similar_texts.add(i)  # Mark text i as similar to idx

# Display the results
print(f"Texts that are similar to others (index-based): {similar_texts}")

# Now filter out similar texts from the original DataFrame
df_unique_texts = df.drop(index=list(similar_texts))

print("\nUnique Texts:")
for text in df_unique_texts['text']:
    print(text)
